# Process plate counts to get ratios of variants on variable cell numbers

The initial re-pool that I tested probably had some technical errors in setup, leading to noisy data. Here, I repeated that dilution series and infection in duplicate (starting with 12.5 uL virus [4x less than previous replicates] and performing 2-fold dilutions) with both my re-pooled H3N2 2023-2024 library, as well as Andrea's pdmH1N1 2021-2022 library. 

The plots generated by this notebook are interactive, so you can mouseover points for details, use the mouse-scroll to zoom and pan, and use interactive dropdowns at the bottom of the plots.

## Setup
Import Python modules:

In [1]:
import pickle
import sys

import altair as alt

import matplotlib.pyplot as plt

import numpy

import pandas as pd
from os.path import join
import os
import ruamel.yaml as yaml

_ = alt.data_transformers.disable_max_rows()

## Add input data locations
Some of these files are defined as data, and some of these files are generated by running the specified library pooling data as `miscellaneous_plates` through the `seqneut-pipeline`. For details on how these files are generated, see the `README.md' in [https://github.com/jbloomlab/seqneut-pipeline](https://github.com/jbloomlab/seqneut-pipeline)

In [104]:
# Define file path prefix
filepath_prefix = '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024'

# Viral library contents and barcode IDs
viral_H3N2_library_csv = filepath_prefix + '/data/viral_libraries/2023_H3N2_Kikawa.csv'
viral_H1N1_library_csv = filepath_prefix + '/data/viral_libraries/pdmH1N1_lib2023_loes.csv'
# Neutralization standard set of barcode IDs
neut_standard_set_csv = filepath_prefix + '/data/neut_standard_sets/loes2023_neut_standards.csv'
# All samples included in library poolign sequencing run
# Contains information on library, dilution factor, R1 location
samplesfile_H3N2 = filepath_prefix + '/data/miscellaneous_plates/2024-03-28_H3N2_MOItest_variableCell.csv'

# Counts and fates files output by running library pooling samples as miscellaneous plates
platedir_H3N2 = filepath_prefix + '/results/miscellaneous_plates/240328_repool_H3N2_variableCell/'
platedir_H3N2 = '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/'

# Identify all counts and fates CSVs
count_csvs = []
fate_csvs = []
file_list_H3N2 = os.listdir(platedir_H3N2)

for f in file_list_H3N2:
    location = platedir_H3N2 + f
    if "_counts" in f:
        count_csvs.append(location)
    elif "_fates" in f:
        fate_csvs.append(location)

fate_csvs

['/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/B1_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/B5_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/G1_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/C1_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/C4_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024/results/miscellaneous_plates/240328_repool_H3N2_variableCell/H3_fates.csv',
 '/fh/fast/bloom_j/computational_notebooks/cki

In [105]:
# Define a samples dataframe using the samples file
samples_df = pd.read_csv(samplesfile_H3N2).reset_index(drop=True)
samples_df.drop(columns=['fastq'], inplace=True)
samples_df['sample'] = samples_df.apply(
    lambda x: '-'.join(x.astype(str)), axis=1
)

samples = samples_df["sample"].unique().tolist()
print(f"There are {len(samples)} barcode runs.")

samples_df

samples

There are 64 barcode runs.


['A1-2023rePool_H3N2_50k-4-1',
 'B1-2023rePool_H3N2_50k-8-1',
 'C1-2023rePool_H3N2_50k-16-1',
 'D1-2023rePool_H3N2_50k-32-1',
 'E1-2023rePool_H3N2_50k-64-1',
 'F1-2023rePool_H3N2_50k-128-1',
 'G1-2023rePool_H3N2_50k-256-1',
 'H1-2023rePool_H3N2_50k-512-1',
 'A2-2023rePool_H3N2_50k-4-2',
 'B2-2023rePool_H3N2_50k-8-2',
 'C2-2023rePool_H3N2_50k-16-2',
 'D2-2023rePool_H3N2_50k-32-2',
 'E2-2023rePool_H3N2_50k-64-2',
 'F2-2023rePool_H3N2_50k-128-2',
 'G2-2023rePool_H3N2_50k-256-2',
 'H2-2023rePool_H3N2_50k-512-2',
 'A3-2023rePool_H3N2_75k-4-1',
 'B3-2023rePool_H3N2_75k-8-1',
 'C3-2023rePool_H3N2_75k-16-1',
 'D3-2023rePool_H3N2_75k-32-1',
 'E3-2023rePool_H3N2_75k-64-1',
 'F3-2023rePool_H3N2_75k-128-1',
 'G3-2023rePool_H3N2_75k-256-1',
 'H3-2023rePool_H3N2_75k-512-1',
 'A4-2023rePool_H3N2_75k-4-2',
 'B4-2023rePool_H3N2_75k-8-2',
 'C4-2023rePool_H3N2_75k-16-2',
 'D4-2023rePool_H3N2_75k-32-2',
 'E4-2023rePool_H3N2_75k-64-2',
 'F4-2023rePool_H3N2_75k-128-2',
 'G4-2023rePool_H3N2_75k-256-2',
 'H4-

## Statistics on barcode-parsing for each sample
Make interactive chart of the "fates" of the sequencing reads parsed for each sample on the plate.

If most sequencing reads are not "valid barcodes", this could potentially indicate some problem in the sequencing or barcode set you are parsing.

Potential fates are:
 - *valid barcode*: barcode that matches a known virus or neutralization standard, we hope most reads are this.
 - *invalid barcode*: a barcode with proper flanking sequences, but does not match a known virus or neutralization standard. If you  have a lot of reads of this type, it is probably a good idea to look at the invalid barcode CSVs (in the `./results/barcode_invalid/` subdirectory created by the pipeline) to see what these invalid barcodes are.
 - *unparseable barcode*: could not parse a barcode from this read as there was not a sequence of the correct length with the appropriate flanking sequence.
 - *low quality barcode*: low-quality or `N` nucleotides in barcode, could indicate problem with sequencing.
 - *failed chastity filter*: reads that failed the Illumina chastity filter, if these are reported in the FASTQ (they may not be).

Also, if the number of reads per sample is very uneven, that could indicate that you did not do a good job of balancing the different samples in the Illumina sequencing.

In [116]:
fates = (
    pd.concat([pd.read_csv(f).assign(well=f
                                     .replace(platedir_H3N2, '') # getting weird behavior with .strip() more and more often...
                                     .strip('_fates.csv')
                                    ) for f in fate_csvs])
    .merge(samples_df,
           how = 'left',
           validate="many_to_one", 
           on="well")
    .assign(
        fate_counts=lambda x: x.groupby("fate")["count"].transform("sum"),
        sample_well=lambda x: x["sample"] + " (" + x["well"] + ")",
    )
    .query("fate_counts > 0")  # only keep fates with at least one count
    [['serum', "fate", "count", "well", "sample_well", "dilution_factor"]]
)

assert len(fates) == len(fates.drop_duplicates())


sample_wells = list(
    fates.sort_values(['serum', "dilution_factor"], ascending=True)["sample_well"]
)



fates_chart = (
    alt.Chart(fates)
    .encode(
        alt.X("count", scale=alt.Scale(nice=False, padding=3)),
        alt.Y(
            "sample_well",
            title=None,
            sort=sample_wells,
        ),
        alt.Color("fate", sort=sorted(fates["fate"].unique(), reverse=True)),
        alt.Order("fate", sort="descending"),
        tooltip=fates.columns.tolist(),
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
        title=f"Barcode parsing for initial titering plate",
    )
    .configure_axis(grid=False)
)

fates_chart

alt.Chart(...)

Looks like we got nice coverage across all wells. Previous issues with missing wells at the cDNA synthesis step have resolved?

## Read barcode counts
Read the counts per barcode:

In [117]:
# get barcode counts
counts = (
    pd.concat([pd.read_csv(c).assign(well=c.replace(platedir_H3N2, '').strip('_counts.csv')) for c, s in zip(count_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="well")
    .drop(columns=["replicate"])
    .assign(sample_well=lambda x: x["sample"] + " (" + x["well"] + ")")
)

# classify barcodes as viral or neut standard
barcode_class = pd.concat(
    [
        pd.read_csv(viral_H3N2_library_csv)[["barcode", "strain"]].assign(
            neut_standard=False,
        ),
        pd.read_csv(neut_standard_set_csv)[["barcode"]].assign(
            neut_standard=True,
            strain=pd.NA,
        ),
    ],
    ignore_index=True,
)

# merge counts and classification of barcodes
assert set(counts["barcode"]) == set(barcode_class["barcode"])
counts = (counts
          .merge(barcode_class, on="barcode")
          # .assign(virus_library = lambda x: x['serum'].str.split('_').str[1][0])
         )

assert set(sample_wells) == set(counts["sample_well"])

# define virus library
counts['virus_library'] = counts['serum'].str.split('_', expand=True)[[1]]

## Average counts per barcode in each well

Plot average counts per barcode.
If a sample has inadequate barcode counts, it may not have good enough statistics for accurate analysis, and a QC-threshold is applied:

In [118]:
avg_barcode_counts = (
    counts.groupby(
        ["well", "sample_well"],
        dropna=False,
        as_index=False,
    )
    .aggregate(avg_count=pd.NamedAgg("count", "mean"))
    .assign(
        fails_qc=lambda x: (
            x["avg_count"] < 500
        ),
    )
)

avg_barcode_counts_chart = (
    alt.Chart(avg_barcode_counts)
    .encode(
        alt.X(
            "avg_count",
            title="average barcode counts per well",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("sample_well", sort=sample_wells),
        alt.Color(
            "fails_qc",
            title=f"fails {'min barcode count threshold'=}",
            legend=alt.Legend(titleLimit=500),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if avg_barcode_counts[c].dtype == float else c
            for c in avg_barcode_counts.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=250,
        title=f"Average barcode counts per well for titering plate",
    )
    .configure_axis(grid=False)
)

display(avg_barcode_counts_chart)

# drop wells failing QC
avg_barcode_counts_per_well_drops = list(avg_barcode_counts.query("fails_qc")["well"])

alt.Chart(...)

## Fraction of counts from neutralization standard
Determine the fraction of counts from the neutralization standard in each sample, and make sure this fraction passess the QC threshold.

In [119]:
# Define dataframe from counts with the fraction of neutralization standard calculated
neut_standard_fracs = (
    counts.assign(
        neut_standard_count=lambda x: x["count"] * x["neut_standard"].astype(int)
    )
    .groupby(
        ['virus_library', "well", "sample_well", 'dilution_factor'],
        dropna=False,
        as_index=False,
    )
    .aggregate(
        total_count=pd.NamedAgg("count", "sum"),
        neut_standard_count=pd.NamedAgg("neut_standard_count", "sum"),
    )
    .assign(
        neut_standard_frac=lambda x: x["neut_standard_count"] / x["total_count"],
        fails_qc=lambda x: (
            x["neut_standard_frac"] < 0.001
        ),
    )
)

# Plot as bar chart
chart_h3n2 = (
    alt.Chart(neut_standard_fracs.query('virus_library == "H3N2"'))
    .encode(
        alt.X(
            "neut_standard_frac",
            title="frac counts from neutralization standard per well",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("sample_well", sort=sample_wells),
        alt.Color(
            "fails_qc",
            legend=alt.Legend(titleLimit=500),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if neut_standard_fracs[c].dtype == float else c
            for c in neut_standard_fracs.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=250,
        title=f"Neutralization-standard fracs per well for titering plate, initial pool",
    )
    # .configure_axis(grid=False)
    # .configure_legend(titleLimit=1000)
)

display(chart_h3n2)

# drop wells failing QC
min_neut_standard_frac_per_well_drops = list(
    neut_standard_fracs.query("fails_qc")["well"]
)

neut_standard_fracs['cell_number'] = neut_standard_fracs['sample_well'].str.split('H3N2_').str[1].str.split('-').str[0]
neut_standard_fracs

alt.Chart(...)

,virus_library,well,sample_well,dilution_factor,total_count,neut_standard_count,neut_standard_frac,fails_qc,cell_number
0,H3N2,A1,A1-2023rePool_H3N2_50k-4-1 (A1),4,1632969,27460,0.016816,False,50k
1,H3N2,A2,A2-2023rePool_H3N2_50k-4-2 (A2),4,3145896,55936,0.017781,False,50k
2,H3N2,A3,A3-2023rePool_H3N2_75k-4-1 (A3),4,2692532,39836,0.014795,False,75k
3,H3N2,A4,A4-2023rePool_H3N2_75k-4-2 (A4),4,2503601,43753,0.017476,False,75k
4,H3N2,A5,A5-2023rePool_H3N2_100k-4-1 (A5),4,1761347,22893,0.012997,False,100k
...,...,...,...,...,...,...,...,...,...
59,H3N2,H4,H4-2023rePool_H3N2_75k-512-2 (H4),512,2443415,781676,0.319911,False,75k
60,H3N2,H5,H5-2023rePool_H3N2_100k-512-1 (H5),512,2251614,751003,0.333540,False,100k
61,H3N2,H6,H6-2023rePool_H3N2_100k-512-2 (H6),512,1936360,692783,0.357776,False,100k
62,H3N2,H7,H7-2023rePool_H3N2_150k-512-1 (H7),512,2082790,788348,0.378506,False,150k


In [120]:
# Scatterplot of the same data as above, plotted by dilution factor
scatter = alt.Chart(neut_standard_fracs
                        # .query('cell_number == "50k"')
                        ).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='cell_number',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).properties(
        height=alt.Step(10),
        width=250,
        title=f"Neutralization-standard fracs per well for H3N2 re-pool",
    ).interactive()

scatter

alt.Chart(...)

In [121]:
# Scatterplot of the same data as above, plotted by dilution factor
scatter_50k = alt.Chart(neut_standard_fracs.query('cell_number == "50k"')
                        ).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='fails_qc',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).properties(
        height=alt.Step(10),
        width=250,
        title="50k cells per well",
    ).interactive()

scatter_75k = alt.Chart(neut_standard_fracs.query('cell_number == "75k"')
                        ).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='fails_qc',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).properties(
        height=alt.Step(10),
        width=250,
        title="75k cells per well",
    ).interactive()

scatter_100k = alt.Chart(neut_standard_fracs.query('cell_number == "100k"')
                        ).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='fails_qc',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).properties(
        height=alt.Step(10),
        width=250,
        title="100k cells per well",
    ).interactive()

scatter_150k = alt.Chart(neut_standard_fracs.query('cell_number == "150k"')
                        ).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='fails_qc',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).properties(
        height=alt.Step(10),
        width=250,
        title="150k cells per well",
    ).interactive()

scatter_50k | scatter_75k | scatter_100k | scatter_150k

alt.HConcatChart(...)

## Assessing balancing of strains contained in the library
Viruses were rescued and blind passaged individually. Based on data from my initial equal-volume pool, I re-pooled viruses to try to achieve more equal representation. I will now assess that re-pooling. 

Each of the 3 viral barcodes associated with each strain were pooled prior to rescue, so they cannot be balanced. 

In [122]:
# Get summed barcode counts for all strains across all wells
straincounts_allbarcodes = (counts.query('virus_library == "H3N2"')
                            .groupby(['sample','sample_well','strain','dilution_factor','serum','well',])
                          .sum()
                          .reset_index()
                          .drop(columns = ['sample_well', 'neut_standard', 'barcode', 'serum', 'virus_library'])
                         )

# Get sum of all virus/barcode counts per well
sumperwell = (straincounts_allbarcodes.groupby(['sample','dilution_factor','well'])
              .sum()
              .drop(columns=['strain'])
              .reset_index()
              .rename(columns={'count':'counts_perwell'})
             )

# Merge dataframes and calculate fraction of each well devoted to each strain
merged_df = straincounts_allbarcodes.merge(sumperwell, on=['sample','dilution_factor','well'])
merged_df['fraction_strain'] = merged_df['count'] /merged_df['counts_perwell'] /2

merged_df['cell_number'] = merged_df['sample'].str.split('H3N2_').str[1].str.split('-').str[0]
merged_df

merged_df

,sample,strain,dilution_factor,well,count,counts_perwell,fraction_strain,cell_number
0,A1-2023rePool_H3N2_50k-4-1,A/Abu_Dhabi/6753/2023,4,A1,13722,1605509,0.004273,50k
1,A1-2023rePool_H3N2_50k-4-1,A/Bangkok/P3599/2023,4,A1,19227,1605509,0.005988,50k
2,A1-2023rePool_H3N2_50k-4-1,A/Bangkok/P3755/2023,4,A1,19912,1605509,0.006201,50k
3,A1-2023rePool_H3N2_50k-4-1,A/Bhutan/0006/2023,4,A1,13533,1605509,0.004215,50k
4,A1-2023rePool_H3N2_50k-4-1,A/Bhutan/0845/2023,4,A1,10191,1605509,0.003174,50k
...,...,...,...,...,...,...,...,...
5051,H8-2023rePool_H3N2_150k-512-2,A/Thailand/8/2022,512,H8,157,1130844,0.000069,150k
5052,H8-2023rePool_H3N2_150k-512-2,A/Townsville/68/2023,512,H8,6761,1130844,0.002989,150k
5053,H8-2023rePool_H3N2_150k-512-2,A/Victoria/1033/2023,512,H8,5129,1130844,0.002268,150k
5054,H8-2023rePool_H3N2_150k-512-2,A/Wisconsin/27/2023,512,H8,6337,1130844,0.002802,150k


We now have this fraction of reads devoted to all strains calculated for all wells. However, ideally we should just focus on those wells containing dilutions that we would use for actual neutralization assays. We should choose a set of replicate wells where the fraction of neutralization standard reads begins to increase linearly with the increasing reciprocal dilution factor. See plots above for choosing these wells. 

In [75]:
# Using E1 and A3, corresponding to reciprocal dilution factor = 32, 75k
single_well = merged_df.loc[merged_df['sample'].str.contains('D3-|D4-')]

In [77]:
# Calculate mean fraction strain across both wells
mean_df = single_well.groupby(['strain'])['fraction_strain'].mean().to_frame().rename(columns = {'fraction_strain': 'mean_fraction_strains'}).reset_index()
mean_single_well = single_well.merge(mean_df, on = 'strain', how = 'left')

# calcualte ratios to add for equal pool
num_strains = 75
mean_single_well['ratio_to_add'] = (1/num_strains)/mean_single_well['fraction_strain']
mean_single_well['mean_ratio_to_add'] = (1/num_strains)/mean_single_well['mean_fraction_strains']

mean_single_well['est_tcid50'] = (mean_single_well['mean_fraction_strains']*25000)*75

mean_single_well.head(6)

,sample,strain,dilution_factor,well,count,counts_perwell,fraction_strain,cell_number,mean_fraction_strains,ratio_to_add,mean_ratio_to_add,est_tcid50
0,D3-2023rePool_H3N2_75k-32-1,A/Abu_Dhabi/6753/2023,32,D3,15874,1949309,0.004072,75k,0.004332,3.274636,3.077683,8122.993784
1,D3-2023rePool_H3N2_75k-32-1,A/Bangkok/P3599/2023,32,D3,27962,1949309,0.007172,75k,0.006594,1.859008,2.021970,12364.176713
2,D3-2023rePool_H3N2_75k-32-1,A/Bangkok/P3755/2023,32,D3,27847,1949309,0.007143,75k,0.008424,1.866685,1.582821,15794.581931
3,D3-2023rePool_H3N2_75k-32-1,A/Bhutan/0006/2023,32,D3,13234,1949309,0.003395,75k,0.003618,3.927881,3.685418,6783.490384
4,D3-2023rePool_H3N2_75k-32-1,A/Bhutan/0845/2023,32,D3,15029,1949309,0.003855,75k,0.003706,3.458751,3.597951,6948.399744
5,D3-2023rePool_H3N2_75k-32-1,A/Bhutan/FLU-BTG-00988/2022,32,D3,18933,1949309,0.004856,75k,0.005087,2.745554,2.621272,9537.355559


## Visualize barcode- and strain-level balancing in the current pool

In [78]:
# Plot the current fraction of each strain in the pool
strains_chart = (
    alt.Chart(mean_single_well)
    .encode(
        alt.X(
            "fraction_strain",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("strain"),
        
        tooltip = ['strain', 'fraction_strain', 'est_tcid50'],
    )
).mark_bar(height={"band": 0.85}).properties(
        height=alt.Step(10),
        width=250,
        title="",
    ).properties(
        height = alt.Step(10),
        width = 200,
        title = "Strain representation, repool")

# add veritcal line where we would expect equal representation of all barcodes in pool
expected_line = alt.Chart(
    pd.DataFrame({'x': [1/num_strains]})
).mark_rule(strokeDash = [2,2], strokeWidth = 2).encode(x = 'x')

# plot both barcode counts and expected line
strains_chart + expected_line

alt.LayerChart(...)

In [80]:
# Each barcode fraction across strains
all_barcode_counts = (counts
                      .query('virus_library == "H3N2"')
                      [['strain', 'barcode', 'count', 'well']].dropna()
                     )
single_well_all_barcode_counts = all_barcode_counts[all_barcode_counts['well'].isin(['E1','A3'])]

# Get tidy single well means
tidy_single_well = single_well_all_barcode_counts[['strain','barcode','count']].groupby(['strain', 'barcode']).mean().reset_index()
# Get sums for each strain
strain_sums_df = tidy_single_well.groupby('strain').sum().rename(columns = {'count': 'strain_count_sum'}).reset_index()
# Merge and calculate per strain the fraction represented by each barcode
tidy_single_well = tidy_single_well.merge(strain_sums_df[['strain', 'strain_count_sum']], 
                       on = ['strain'],
                       validate="many_to_one",
                      )
tidy_single_well['per_strain_fraction_barcode'] = tidy_single_well['count'] / tidy_single_well['strain_count_sum']
tidy_single_well['barcode_letter'] = (['A', 'B', 'C'] * len(strain_sums_df))

# Plot as colored bar chart
bar_chart = alt.Chart(tidy_single_well).mark_bar(height={"band": 0.85}).encode(
    x = 'per_strain_fraction_barcode',
    y = 'strain',
    color=alt.Color('barcode_letter', legend=None),
    tooltip = ['strain', 'per_strain_fraction_barcode', 'barcode'],
).configure_axis(grid=False).properties(
        height = alt.Step(10),
        width = 200,
        title = "Barcode fraction for each strain, repool")

bar_chart

alt.Chart(...)

Beyond a few strains with lower representation in this experiment (mostly egg-based vaccine strains), this pools looks fairly well-balanced.